### Cargar drive

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Installing dependencies

In [50]:
!pip install mediapipe face_recognition opencv-python dlib==19.24.2 yolov5

### Python Scripts Path

In [51]:
%cd /content/drive/MyDrive/Equipo 2 - TC3007C/Reto IA II/Modelo Participación y Asistencia
# %cd /content/drive/MyDrive/Modelo Participación y Asistencia

/content/drive/MyDrive/Equipo 2 - TC3007C/Reto IA II/Modelo Participación y Asistencia


### Library import

In [52]:
# Libraries to use
import cv2
import yolov5
import face_recognition
import mediapipe as mp
from datetime import datetime
import traceback
from matplotlib import pyplot as plt

# Auxiliar Scripts
from Person import Person

### Define the students data

In [53]:
data = {
    'person_1' : {
        'id': 1,
        'name': 'Alberto',
        # 'img_url': '/content/drive/MyDrive/Modelo Participación y Asistencia/20231113_110250.jpg',
        'img_url': '/content/drive/MyDrive/Equipo 2 - TC3007C/Reto IA II/Modelo Participación y Asistencia/Alberto.jpg',
    },
    'person_2' : {
        'id': 2,
        'name': 'Luis',
        # 'img_url': '/content/drive/MyDrive/Modelo Participación y Asistencia/20231001_184708.jpg',
        'img_url': '/content/drive/MyDrive/Equipo 2 - TC3007C/Reto IA II/Modelo Participación y Asistencia/Luis.jpg',
    },
    'person_3' : {
        'id': 3,
        'name' : 'Julian',
        # 'img_url': '/content/drive/MyDrive/Modelo Participación y Asistencia/Julian.jpg'
        'img_url': '/content/drive/MyDrive/Equipo 2 - TC3007C/Reto IA II/Modelo Participación y Asistencia/Julian.jpg'
    },
    'person_4' : {
        'id': 4,
        'name' : 'Antonio',
        # 'img_url': '/content/drive/MyDrive/Modelo Participación y Asistencia/Julian.jpg'
        'img_url': '/content/drive/MyDrive/Equipo 2 - TC3007C/Reto IA II/Modelo Participación y Asistencia/Antonio.jpg'
    }

    # 'person_4' : {
    #     'id': 4,
    #     'name': 'Alonso',
    #     # 'img_url': '/content/drive/MyDrive/Modelo Participación y Asistencia/20230805_113046.jpg'
    #     'img_url': '/content/drive/MyDrive/Equipo 2 - TC3007C/Reto IA II/Modelo Participación y Asistencia/20230805_113046.jpg',
    # }
}

### Create arrays to store people, face names and face encodings

In [54]:
people = []
known_face_encodings = []

### Initialize an object Person to keep track of attendance and participation

In [55]:
for person_key, person_info in data.items():
    name = person_info['name']
    person_id = person_info['id']
    img_url = person_info['img_url']

    person = Person(name, person_id, img_url)
    people.append(person)
    known_face_encodings.append(person.person_face_encoding)

### Initialize Pose Model

In [56]:
pose = mp.solutions.pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.9)

### Load the YOLOv5 model

In [57]:
model_path = 'yolov5s.pt'
# device = "cpu"  # for cpu
device = 0  # for gpu
yolov5 = yolov5.YOLOv5(model_path, device, load_on_init=True)

### Path to the input MP4 file, Creation of the VideoCapture Object and Ouput file path

In [58]:
# Input file path
mp4_input_file_path = '/content/drive/MyDrive/Equipo 2 - TC3007C/Reto IA II/Modelo Participación y Asistencia/input_5.mp4'
# mp4_input_file_path = '/content/drive/MyDrive/Modelo Participación y Asistencia/input.mp4'
# Create a VideoCapture object using the MP4 file
video_capture = cv2.VideoCapture(mp4_input_file_path)
# Output filepath
mp4_output_file_path = '/content/drive/MyDrive/Equipo 2 - TC3007C/Reto IA II/Modelo Participación y Asistencia/output_video_10.mp4'
# mp4_output_file_path = '/content/drive/MyDrive/Modelo Participación y Asistencia/output_video_7.mp4'

### Define the codec and VideoWriter to generate an output MP4 file

In [59]:
# Get the video frame width, height, and frames per second information
width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video_capture.get(cv2.CAP_PROP_FPS))

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'XVID' for AVI or 'mp4v' for MP4
out = cv2.VideoWriter(mp4_output_file_path, fourcc, fps, (width, height))

### While loop for the facial and pose recognition

In [60]:
# Process each frame of the video
while True:
    # Read the next frame
    success, frame = video_capture.read()

    if frame is None:
      print("Frame is None. Exiting.")
      break

    # Find all face locations and face encodings in the current frame
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    # Process each face in the frame
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # Check if the face matches any of the known persons
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.5)
        name = "Unknown"

        if True in matches:
            first_match_index = matches.index(True)
            person = people[first_match_index]
            name = person.name

            if person.name != "Unknown" and not person.assistance:
                # Log attendance
                with open("attendance_log.txt", "a") as log_file:
                    log_file.write(f"{name} - {datetime.now()}\n")

                # Mark attendance as taken
                person.assistance = True

        # Draw rectangle around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw label
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

    # Perform object detection on the frame
    results = yolov5.predict(frame, size=640, augment=False)
    detections = results.pred[0]

    # Check whether the bounding box centroids are inside the ROI
    for detection in detections:
        xmin = detection[0]
        ymin = detection[1]
        xmax = detection[2]
        ymax = detection[3]
        score = detection[4]
        class_id = detection[5]
        centroid_x = int(xmin + xmax) // 2
        centroid_y = int(ymin + ymax) // 2

        # Threshold score
        if score >= 0.6:
            if class_id == 0:
                color = (255, 0, 0)
                cv2.rectangle(frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), color, 1)

                # Padding
                padding = 25
                person_roi = frame[int(ymin):int(ymax), int(xmin):int(xmax)]

                try:  # Mediapipe
                    # Count arm raises
                    results_pose = pose.process(person_roi)
                    if results_pose.pose_landmarks:

                        # Shoulder landmarks (left and right)
                        left_shoulder = results_pose.pose_landmarks.landmark[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER]
                        right_shoulder = results_pose.pose_landmarks.landmark[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER]

                        # Hand landmarks (for example, the tip of the index finger)
                        left_hand = results_pose.pose_landmarks.landmark[mp.solutions.pose.PoseLandmark.LEFT_INDEX]
                        right_hand = results_pose.pose_landmarks.landmark[mp.solutions.pose.PoseLandmark.RIGHT_INDEX]

                        # Pose tracking logic (same as in your original script)
                        left_arm_landmarks = results_pose.pose_landmarks.landmark[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER:mp.solutions.pose.PoseLandmark.LEFT_WRIST]
                        right_arm_landmarks = results_pose.pose_landmarks.landmark[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER:mp.solutions.pose.PoseLandmark.RIGHT_WRIST]

                        left_arm_average_y = sum(point.y for point in left_arm_landmarks) / len(left_arm_landmarks)
                        right_arm_average_y = sum(point.y for point in right_arm_landmarks) / len(right_arm_landmarks)

                        left_arm_up = left_arm_average_y < left_arm_landmarks[0].y
                        right_arm_up = right_arm_average_y < right_arm_landmarks[0].y

                        # Check if the person is participating based on hand positions
                        not_participating = left_hand.y < left_shoulder.y and right_hand.y < right_shoulder.y

                        rgb_roi = cv2.cvtColor(person_roi, cv2.COLOR_BGR2RGB)

                        # cv2.imshow("Detected participation", rgb_roi)
                        # cv2.waitKey(0)

                        if rgb_roi.any():
                            rgb_encoding = face_recognition.face_encodings(rgb_roi)[0]

                            # print(f"left hand: {left_hand}, right_hand: {right_hand}")

                            matches = face_recognition.compare_faces(known_face_encodings, rgb_encoding, tolerance=0.48)

                            if True in matches:
                                idx = matches.index(True)

                                if people[idx].arm_raised == False and (left_arm_up or right_arm_up) and not not_participating:
                                        people[idx].participations += 1
                                        people[idx].arm_raised = True
                                        print(f"Participation Registered for {people[idx].name}")

                                elif people[idx].arm_raised == True and not (left_arm_up or right_arm_up) and not_participating:
                                        print(f"{people[idx].name}'s arm down")
                                        people[idx].arm_raised = False

                except Exception as e:
                    print(f"Error processing image: {e}")
                    print(traceback.format_exc())

            else:
                pass

    # Display the frame with counters for each person
    for idx, person in enumerate(people):
        cv2.putText(frame, f"{person.name}: {person.participations} participations", (10, 30 + 20 * idx),
            cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 0, 0), 1)

    # Display the frame
    # cv2.imshow("Video", frame)

    out.write(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object
video_capture.release()
out.release()

cv2.destroyAllWindows()

Error processing image: list index out of range
Traceback (most recent call last):
  File "<ipython-input-60-b763a209dd3b>", line 97, in <cell line: 2>
    rgb_encoding = face_recognition.face_encodings(rgb_roi)[0]
IndexError: list index out of range

Error processing image: list index out of range
Traceback (most recent call last):
  File "<ipython-input-60-b763a209dd3b>", line 97, in <cell line: 2>
    rgb_encoding = face_recognition.face_encodings(rgb_roi)[0]
IndexError: list index out of range

Error processing image: list index out of range
Traceback (most recent call last):
  File "<ipython-input-60-b763a209dd3b>", line 97, in <cell line: 2>
    rgb_encoding = face_recognition.face_encodings(rgb_roi)[0]
IndexError: list index out of range

Error processing image: list index out of range
Traceback (most recent call last):
  File "<ipython-input-60-b763a209dd3b>", line 97, in <cell line: 2>
    rgb_encoding = face_recognition.face_encodings(rgb_roi)[0]
IndexError: list index out of